<a href="https://colab.research.google.com/github/FFI-Vietnam/camtrap-tools/blob/main/Wildlife%20Insights/bulk-upload/01_clean-exiftool-dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
This script is used for cleaning the metadata spreadsheet generated by ExifTool
After runnning this script, 'data cleaning' folder is created

data cleaning
    |__ 01_metadata.csv
    |__ 02_metadata_new-column.csv
    |__ 03_metadata_single-species-row.csv
    |__ 04_metadata_remove-blank.csv
    
"""

In [5]:
import pandas as pd
import numpy as np
import os

In [2]:
# mount with Google Drive to read and save file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# species Colab directories
root = '/content/drive/'
contain_folder = 'My Drive/FFI/Wildlife Insights Bulk Upload Test/bulk-upload_template-autofill/dataset'
file_name = 'image_metadata(2020-06-26)_full.csv'

In [64]:
def save_csv_Google_drive(df, root, contain_folder, file_name):
  """
  function to save a csv file to Google Drive
  param examples:
    root = '/content/drive/'
    contain_folder = 'My Drive/FFI/dataset'
    file_name = 'image_metadata(2020-06-26)_full.csv'
  """
  # save file to Colab runtime storage (will be deleted when this notebook is closed)
  df.to_csv('dataframe.csv', index=False)

  # save file back to Google Drive for permanent storage
  folder_path = os.path.join(root, contain_folder)
  file_path = os.path.join(root, contain_folder, file_name)
  try:
    os.makedirs(folder_path)
  except:
    pass

  with open('dataframe.csv', 'r') as f:
    df_file = f.read()

  with open(file_path, 'w') as f:
    f.write(df_file)

  print(f'File is saved to {file_name} in Google Drive at {file_path}')

In [31]:
# read full dataset
file_path = os.path.join(root, contain_folder, file_name)
full_metadata = pd.read_csv(file_path)
full_metadata.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,20,41,42,45,56,57,58,60,61,75) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SourceFile,BitsPerSample,Caption-Abstract,CatalogSets,Categories,CodedCharacterSet,ColorComponents,ColorLabel,Comment,CreateDate,CreatorTool,CurrentIPTCDigest,DateCreated,DateTime,DateTimeCreated,DateTimeOriginal,DateUTC,Description,Directory,EncodingProcess,Error,ExifByteOrder,ExifImageHeight,ExifImageWidth,ExifToolVersion,ExifVersion,FileAccessDate,FileCreateDate,FileModifyDate,FileName,FilePermissions,FileSize,FileType,FileTypeExtension,Flash,HierarchicalSubject,ImageDescription,ImageHeight,ImageNumber,ImageSize,ImageWidth,InstanceID,IPTCDigest,JFIFVersion,Keywords,LastKeywordIPTC,LastKeywordXMP,Make,Megapixels,MetadataDate,MIMEType,Model,ModificationDate,ModifyDate,Notes,OffsetSchema,Orientation,OriginatingProgram,Padding,PickLabel,ProcessingSoftware,ProgramVersion,Rating,RatingPercent,RegionInfoRegions,RegionList,ResolutionUnit,Software,Subject,TagsList,TimeCreated,Urgency,UserComment,Warning,XMPToolkit,XPKeywords,XResolution,YCbCrSubSampling,YResolution
0,D:/Fauna & Flora International/VietnamCameraSt...,8.0,NaN,"People|Setup, Observer|Tam","<Categories><Category Assigned=""0"">People<Cate...",UTF8,3.0,NaN,NaN,2019:04:22 15:09:11,NaN,0dfadea9315c091a67a43eb6881ea5df,NaN,NaN,NaN,2019:04:22 15:09:11,NaN,NaN,D:/Fauna & Flora International/VietnamCameraSt...,"Baseline DCT, Huffman coding",NaN,"Big-endian (Motorola, MM)",1536.0,2048.0,11.63,220.0,2020:05:26 10:18:20+07:00,2020:03:20 13:23:52+07:00,2020:05:07 10:40:30+07:00,100__67096__2019-04-22__15-09-11(1).JPG,rw-rw-rw-,458 kB,JPEG,jpg,Fired,"People|Setup, Observer|Tam",integtime=1535 A=88 D=128\r\n\rflashCrossover...,1536.0,1154.0,2048x1536,2048.0,NaN,NaN,1.02,"Setup, Tam",NaN,"People/Setup, Observer/Tam",Panthera V4,3.1,NaN,image/jpeg,CAM67096,NaN,2019:04:22 15:09:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,102717 V6-Bld1.15,"Setup, Tam","People/Setup, Observer/Tam",NaN,NaN,NaN,[minor] Unrecognized MakerNotes,XMP Core 4.4.0-Exiv2,NaN,1.0,YCbCr4:2:2 (2 1),1.0
1,D:/Fauna & Flora International/VietnamCameraSt...,8.0,NaN,"People|Setup, Observer|Tam","<Categories><Category Assigned=""0"">People<Cate...",UTF8,3.0,NaN,NaN,2019:04:22 15:09:42,NaN,0dfadea9315c091a67a43eb6881ea5df,NaN,NaN,NaN,2019:04:22 15:09:42,NaN,NaN,D:/Fauna & Flora International/VietnamCameraSt...,"Baseline DCT, Huffman coding",NaN,"Big-endian (Motorola, MM)",1536.0,2048.0,11.63,220.0,2020:06:24 14:00:28+07:00,2020:03:20 13:23:52+07:00,2020:05:07 10:40:30+07:00,100__67096__2019-04-22__15-09-42(2).JPG,rw-rw-rw-,471 kB,JPEG,jpg,Fired,"People|Setup, Observer|Tam",integtime=1535 A=88 D=128\r\n\rflashCrossover...,1536.0,1155.0,2048x1536,2048.0,NaN,NaN,1.02,"Setup, Tam",NaN,"People/Setup, Observer/Tam",Panthera V4,3.1,NaN,image/jpeg,CAM67096,NaN,2019:04:22 15:09:42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,102717 V6-Bld1.15,"Setup, Tam","People/Setup, Observer/Tam",NaN,NaN,NaN,[minor] Unrecognized MakerNotes,XMP Core 4.4.0-Exiv2,NaN,1.0,YCbCr4:2:2 (2 1),1.0
2,D:/Fauna & Flora International/VietnamCameraSt...,8.0,NaN,"People|Setup, Observer|Tam","<Categories><Category Assigned=""0"">People<Cate...",UTF8,3.0,NaN,NaN,2019:04:22 15:12:32,NaN,0dfadea9315c091a67a43eb6881ea5df,NaN,NaN,NaN,2019:04:22 15:12:32,NaN,NaN,D:/Fauna & Flora International/VietnamCameraSt...,"Baseline DCT, Huffman coding",NaN,"Big-endian (Motorola, MM)",1536.0,2048.0,11.63,220.0,2020:06:24 14:00:29+07:00,2020:03:20 13:23:52+07:00,2020:05:07 10:40:30+07:00,100__67096__2019-04-22__15-12-32(1).JPG,rw-rw-rw-,485 kB,JPEG,jpg,Fired,"People|Setup, Observer|Tam",integtime=1535 A=88 D=128\r\n\rflashCrossover...,1536.0,1155.0,2048x1536,2048.0,NaN,NaN,1.02,"Setup, Tam",NaN,"People/Setup, Observer/Tam",Panthera V4,3.1,NaN,image/jpeg,CAM67096,NaN,2019:04:22 15:12:32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,102717 V6-Bld1.15,"Setup, Tam","People/Setup, Observer/Tam",NaN,NaN,NaN,[minor] Unrecognized MakerNotes,XMP Core 4.4.0-Exiv2,NaN,1.0,YCbCr4:2:2 (2 1),1.0
3,D:/Fauna & Flora International/VietnamCameraSt...,8.0,NaN,"People|Se

In [65]:
# 01) keep important columns
metadata = full_metadata[['SourceFile', 'Directory', 'FileName', 'DateTimeOriginal', 'CatalogSets', 'Make', 'Model']]

# save to metadata.csv in Google Drive
save_csv_Google_drive(metadata, root, "My Drive/FFI/Wildlife Insights Bulk Upload Test/bulk-upload_template-autofill/data cleaning", "01_metadata.csv")

metadata.head()

File is saved to 01_metadata.csv in Google Drive at /content/drive/My Drive/FFI/Wildlife Insights Bulk Upload Test/bulk-upload_template-autofill/data cleaning/01_metadata.csv


,SourceFile,Directory,FileName,DateTimeOriginal,CatalogSets,Make,Model
0,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-11(1).JPG,2019:04:22 15:09:11,"People|Setup, Observer|Tam",Panthera V4,CAM67096
1,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-42(2).JPG,2019:04:22 15:09:42,"People|Setup, Observer|Tam",Panthera V4,CAM67096
2,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-32(1).JPG,2019:04:22 15:12:32,"People|Setup, Observer|Tam",Panthera V4,CAM67096
3,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-55(2).JPG,2019:04:22 15:12:55,"People|Setup, Observer|Tam",Panthera V4,CAM67096
4,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-13-18(1).JPG,2019:04:22 15:13:18,"People|Setup, Observer|Tam",Panthera V4,CAM67096


In [66]:
# 02) create new columns
"""
- certainty: an id is certain or not, follow WI guideline. Values = ['Absolutely sure', 'Not  sure']
- species_common_name: extract from tag 'Species' in 'CatalogSets'.
- observer: extract from tag 'Observer' in 'CatalogSets'.
- uploaded: specify if this image has already been uploaded to WI ('x' means YES, empty means NO)
- station: the station of the camera
- camera_id: the camera that took this image
"""
metadata_new_column = metadata.copy()

certainty = [] 
species_common_name = [] 
observer = []
people = []
uploaded = []
station = []
camera_id = []

UNCERTAIN = 'Uncertain Identification'
SPECIES_PREFIX = 'Species'
OBSERVER_PREFIX = 'Observer'
PEOPLE_PREFIX = 'People'

# Create certainty, species_common_name, observer, and people columns
catalog_sets = metadata['CatalogSets']
for i in range(len(catalog_sets)):
  # split the tags
  if catalog_sets[i] is not np.NaN:
    # specify the image is not uploaded
    uploaded.append('')
    catalog = str(catalog_sets[i]).split(', ')
    # specify uncertainty
    if UNCERTAIN in catalog:
      certainty.append('Not sure')
    else:
      certainty.append('Absolutely sure')
    spe = '' # for species
    obs = '' # for observer
    for i in range(len(catalog)): # go through each tag
      if SPECIES_PREFIX in catalog[i]:
        if catalog[i].split('|')[1] != 'Human':
          if '(unknown)' in catalog[i].split('|')[1]:
            # remove (unknown) mark
            spe = spe + ', ' + catalog[i].split('|')[1].split('(unknown)')[0]
          else:
            spe = spe + ', ' + catalog[i].split('|')[1]
      # Human is a bit tricky, it either has 'People' or 'Human' prefix, or both, 
      # so have to check to avoid duplication
      if PEOPLE_PREFIX in catalog[i]:
        if not 'Human' in spe:
          spe = spe + ', ' + 'Human'
      # specify observer
      if OBSERVER_PREFIX in catalog[i]:
        obs = obs + ', ' + catalog[i].split('|')[1]

    species_common_name.append(spe[2:]) # eliminate leading ', '
    observer.append(obs[2:])

  else:
    uploaded.append('')
    certainty.append('')
    observer.append('')
    species_common_name.append('')

# Create station and camera_id column
for file in metadata_new_column['FileName'].to_list():
  station.append(file.split('__')[0])
  camera_id.append(file.split('__')[1])

metadata_new_column = metadata.copy()
metadata_new_column['certainty'] = certainty
metadata_new_column['species_common_name'] = species_common_name
metadata_new_column['observer'] = observer
metadata_new_column['uploaded'] = uploaded
metadata_new_column['station'] = station
metadata_new_column['camera_id'] = camera_id

# save file to 02_metadata_new-column.csv in Google Drive
save_csv_Google_drive(metadata_new_column, root, "My Drive/FFI/Wildlife Insights Bulk Upload Test/bulk-upload_template-autofill/data cleaning", "02_metadata_new-column.csv")

metadata_new_column.head(5)

File is saved to 02_metadata_new-column.csv in Google Drive at /content/drive/My Drive/FFI/Wildlife Insights Bulk Upload Test/bulk-upload_template-autofill/data cleaning/02_metadata_new-column.csv


,SourceFile,Directory,FileName,DateTimeOriginal,CatalogSets,Make,Model,certainty,species_common_name,observer,uploaded,station,camera_id
0,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-11(1).JPG,2019:04:22 15:09:11,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,,100,67096
1,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-42(2).JPG,2019:04:22 15:09:42,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,,100,67096
2,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-32(1).JPG,2019:04:22 15:12:32,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,,100,67096
3,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-55(2).JPG,2019:04:22 15:12:55,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,,100,67096
4,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-13-18(1).JPG,2019:04:22 15:13:18,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,,100,67096


In [61]:
# 03) explode species_common_name column so that each row contains at most one 
# species, i.e. if one image has more than one species, it will be exploded into
# multiple rows

metadata_single_species_row = metadata_new_column.copy()

# set the columns not to be touched as the index
col = metadata_single_species_row.columns.to_list()
col.remove('species_common_name')

# split on comma to get a column of lists, then call explode to explode the 
# list values into their own rows, then reset the index
metadata_single_species_row = metadata_single_species_row.set_index(col).apply(lambda x: x.str.split(',').explode()).reset_index()

# strip ' ' at both ends
metadata_single_species_row['species_common_name'] = metadata_single_species_row['species_common_name'].str.strip()

# save file to 03_metadata_single-species-row.csv in Google Drive
save_csv_Google_drive(metadata_single_species_row, root, "My Drive/FFI/Wildlife Insights Bulk Upload Test/bulk-upload_template-autofill/data cleaning", "03_metadata_single-species-row.csv")

metadata_single_species_row.head(5)

File is saved to 03_metadata_single-species-row.csv in Google Drive at /content/drive/My Drive/FFI/Wildlife Insights Bulk Upload Test/bulk-upload_template-autofill/data cleaning/03_metadata_single-species-row.csv


,SourceFile,Directory,FileName,DateTimeOriginal,CatalogSets,Make,Model,certainty,observer,uploaded,station,camera_id,species_common_name
0,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-11(1).JPG,2019:04:22 15:09:11,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Tam,,100,67096,Human
1,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-42(2).JPG,2019:04:22 15:09:42,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Tam,,100,67096,Human
2,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-32(1).JPG,2019:04:22 15:12:32,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Tam,,100,67096,Human
3,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-55(2).JPG,2019:04:22 15:12:55,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Tam,,100,67096,Human
4,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-13-18(1).JPG,2019:04:22 15:13:18,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Tam,,100,67096,Human


In [62]:
# 04) remove empty images

metadata_remove_blank = metadata_single_species_row.copy()
metadata_remove_blank['species_common_name'].replace('', np.nan, inplace=True)
metadata_remove_blank.dropna(subset=['species_common_name'], how = 'all', inplace=True)

# save file to 04_metadata_remove-blank.csv in Google Drive
save_csv_Google_drive(metadata_remove_blank, root, "My Drive/FFI/Wildlife Insights Bulk Upload Test/bulk-upload_template-autofill/data cleaning", "04_metadata_remove-blank.csv")

metadata_remove_blank.head(5)

File is saved to 04_metadata_remove-blank.csv in Google Drive at /content/drive/My Drive/FFI/Wildlife Insights Bulk Upload Test/bulk-upload_template-autofill/data cleaning/04_metadata_remove-blank.csv


,SourceFile,Directory,FileName,DateTimeOriginal,CatalogSets,Make,Model,certainty,observer,uploaded,station,camera_id,species_common_name
0,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-11(1).JPG,2019:04:22 15:09:11,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Tam,,100,67096,Human
1,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-42(2).JPG,2019:04:22 15:09:42,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Tam,,100,67096,Human
2,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-32(1).JPG,2019:04:22 15:12:32,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Tam,,100,67096,Human
3,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-55(2).JPG,2019:04:22 15:12:55,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Tam,,100,67096,Human
4,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-13-18(1).JPG,2019:04:22 15:13:18,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Tam,,100,67096,Human
